In [2]:
import numpy as np

import gittins

# 17.12

In [3]:
def thompson_simulation(probabilities, payouts, T, beta):
    """Algorithm 17.1"""
    # TODO: Incorporate payouts (in problem 17.16)

    # Init
    n = len(probabilities)
    a = np.ones(n)
    b = np.ones(n)
    
    X = np.random.random(T)
    traj = np.zeros(T)
    rewards = np.zeros(T)

    for i in range(T):
        draw = np.random.beta(a, b)
        idx = np.argmax(draw)

        if X[i] <= probabilities[idx]:
            a[idx] += 1
            traj[i] = traj[i-1] + 1
            rewards[i] = payouts[idx]
        else:
            b[idx] += 1
            traj[i] = traj[i-1]

    # beta array
    B = beta**np.arange(T)

    return np.sum(rewards*B), traj/np.arange(1, T+1)

# True probabilities; payouts
PROBS = np.array([0.2, 0.5, 0.7])
PAYOUTS = np.array([1,1,1])
T = 100
beta = 0.9

thompson_simulation(PROBS, PAYOUTS, T, beta)

(3.7799229979569557,
 array([0.        , 0.        , 0.33333333, 0.5       , 0.4       ,
        0.33333333, 0.28571429, 0.25      , 0.22222222, 0.2       ,
        0.27272727, 0.33333333, 0.38461538, 0.35714286, 0.33333333,
        0.375     , 0.41176471, 0.38888889, 0.36842105, 0.4       ,
        0.42857143, 0.45454545, 0.43478261, 0.41666667, 0.44      ,
        0.46153846, 0.48148148, 0.5       , 0.48275862, 0.5       ,
        0.48387097, 0.5       , 0.48484848, 0.47058824, 0.45714286,
        0.44444444, 0.45945946, 0.47368421, 0.48717949, 0.5       ,
        0.51219512, 0.52380952, 0.51162791, 0.5       , 0.51111111,
        0.52173913, 0.53191489, 0.54166667, 0.53061224, 0.54      ,
        0.54901961, 0.53846154, 0.54716981, 0.55555556, 0.56363636,
        0.57142857, 0.56140351, 0.55172414, 0.55932203, 0.56666667,
        0.55737705, 0.56451613, 0.57142857, 0.578125  , 0.56923077,
        0.57575758, 0.58208955, 0.58823529, 0.5942029 , 0.58571429,
        0.57746479, 0.58333

In [4]:
def thompson_v_gittins_sim(T=100):
    """Simulate three-armed bandit process with
    all payouts equal to 1 for T iterations
    using both Thompson sampling and Gittins index.

    Return:
        totals (ndarray): [thompson_total, gittins_total]
    """

    # True probabilities; payouts
    PROBS = np.array([0.2, 0.5, 0.7])
    PAYOUTS = np.array([1,1,1])

    M = 200
    K = 19
    beta = 0.9

    thompson_total = thompson_simulation(PROBS, PAYOUTS, T, beta)[0]
    gittins_total = gittins.simulation(PROBS, PAYOUTS, K, T, M, beta)[0]

    return np.array([thompson_total, gittins_total])

def thompson_v_gittins_sim_repeat(num_repeats=20, T=100):
    """Average 20 simulations of three-armed bandit processes for T iterations.

    Return:
        averages (ndarray): [thompson_average, gittins_average]
    """

    return np.mean([thompson_v_gittins_sim(T) for _ in range(num_repeats)], axis=0)
    
thompson_v_gittins_sim_repeat(), thompson_v_gittins_sim_repeat(T=50), thompson_v_gittins_sim_repeat(T=200)

(array([5.35545677, 6.09356155]),
 array([5.32245149, 6.0573707 ]),
 array([5.49953075, 5.81331903]))

# 17.13

In [5]:
def AB_testing(probabilities, payouts, m, T):
    """Test each of the n arms m times to estimate probabilities.
    Then pull arm with highest probabilities for remaining T-nm pulls.

    Return:
        total reward
    """

    n = len(probabilities)

    # Pull each arm m times
    AB_pulls = np.random.binomial(m, probabilities, n)

    # TODO: incorporate payouts (so use arm with highest expected value)
    # as well as for rewards from AB_pulls and pulls

    # Choose arm for remaining T-nm pulls
    arm_idx = np.argmax(AB_pulls)

    pulls = np.random.binomial(T-n*m, probabilities[arm_idx])
    
    return np.sum(AB_pulls) + pulls

In [6]:
# Compare with beta=1

# True probabilities; payouts
PROBS = np.array([0.2, 0.5, 0.7])
PAYOUTS = np.array([1,1,1])
m = 10
T = 100
beta = 1

thompson_simulation(PROBS, PAYOUTS, T, beta)[0], AB_testing(PROBS, PAYOUTS, m, T)

(63.0, 60)

# 17.14

In [7]:
def random_testing(probabilities, payouts, m, T):
    """Test each of the n arms m times to estimate probabilities.
    Then pull arm with highest probabilities for remaining T-nm pulls.

    Return:
        total reward
    """

    n = len(probabilities)

    # Count how many times to sample each arm
    choices = np.random.choice(n, m, replace=True)
    counts = np.unique(choices, return_counts=True)[1].astype(int)

    # Pull each arm according to the counts found above
    random_pulls = np.random.binomial(counts, probabilities, n)
    # return random_pulls, counts, random_pulls/counts
    
    # TODO: incorporate payouts (so use arm with highest expected value)
    # as well as for rewards from AB_pulls and pulls

    # Choose arm for remaining T-nm pulls
    arm_idx = np.argmax(random_pulls/counts)

    pulls = np.random.binomial(T-m, probabilities[arm_idx])
    return np.sum(random_pulls) + pulls

In [8]:
# Compare with beta=1

# True probabilities; payouts
PROBS = np.array([0.2, 0.5, 0.7])
PAYOUTS = np.array([1,1,1])
m = 10
T = 100
beta = 1

thompson_simulation(PROBS, PAYOUTS, T, beta)[0], AB_testing(PROBS, PAYOUTS, m, T), random_testing(PROBS, PAYOUTS, m, T)

(69.0, 63, 42)

# 17.15

In [38]:
def discounted_AB_testing(probabilities, payouts, m, T, beta):
    """Test each of the n arms m times to estimate probabilities.
    Then pull arm with highest probabilities for remaining T-nm pulls.

    Return:
        total reward
    """

    n = len(probabilities)

    AB_pulls = np.zeros(n)
    total_reward = 0

    # beta array
    B = beta**np.arange(T)

    # Pull each arm m times in random order
    pulls = np.random.choice(np.concatenate([i * np.ones(m, dtype=int) for i in range(3)]), n*m, replace=False)
    for i, arm_prob in enumerate(probabilities):
        # Count the sucessess
        sequence = np.random.choice(2, m, replace=True, p=np.array([1-arm_prob, arm_prob]))
        AB_pulls[i] = np.count_nonzero(sequence)

        # Add reward, using random order of pulls to determine beta exponent
        pull_indices = (pulls == i)
        total_reward += np.sum(B[:m*n][pull_indices] * sequence)

    # TODO: incorporate payouts (so use arm with highest expected value)
    # as well as for rewards from AB_pulls and pulls

    # Choose arm for remaining T-nm pulls
    arm_idx = np.argmax(AB_pulls)

    # Pull arm
    arm_prob = probabilities[arm_idx]
    sequence = np.random.choice(2, T-n*m, replace=True, p=np.array([1-arm_prob, arm_prob]))

    # Add reward
    total_reward += np.sum(B[n*m:] * sequence)
    
    return total_reward

In [43]:
def discounted_thompson_v_AB_sim(T=100):
    """Simulate three-armed bandit process with
    all payouts equal to 1 for T iterations
    using both Thompson sampling and AB testing.

    Return:
        totals (ndarray): [thompson_total, AB_total]
    """

    # True probabilities; payouts
    PROBS = np.array([0.2, 0.5, 0.7])
    PAYOUTS = np.array([1,1,1])

    beta = 0.9
    m = 10

    thompson_total = thompson_simulation(PROBS, PAYOUTS, T, beta)[0]
    AB_total = discounted_AB_testing(PROBS, PAYOUTS, m, T, beta)

    return np.array([thompson_total, AB_total])

def discounted_thompson_v_AB_sim_repeat(num_repeats=20, T=100):
    """Average 20 simulations of three-armed bandit processes for T iterations.

    Return:
        averages (ndarray): [thompson_average, AB_average]
    """

    return np.mean([discounted_thompson_v_AB_sim(T) for _ in range(num_repeats)], axis=0)

discounted_thompson_v_AB_sim_repeat()

array([5.40238521, 4.67063899])

# 17.16

In [45]:
def thompson(probabilities, payouts, T, beta):
    """Algorithm 17.1"""

    # Init
    n = len(probabilities)
    a = np.ones(n)
    b = np.ones(n)
    
    X = np.random.random(T)
    traj = np.zeros(T)
    rewards = np.zeros(T)

    for i in range(T):
        draw = np.random.beta(a, b)
        idx = np.argmax(payouts*draw)

        if X[i] <= probabilities[idx]:
            a[idx] += 1
            traj[i] = traj[i-1] + 1
            rewards[i] = payouts[idx]
        else:
            b[idx] += 1
            traj[i] = traj[i-1]

    # beta array
    B = beta**np.arange(T)

    return np.sum(rewards*B), traj/np.arange(1, T+1)

# True probabilities; payouts
PROBS = np.array([0.2, 0.5, 0.7])
PAYOUTS = np.array([1.5, 1, 0.5])
T = 1000
beta = 0.99

thompson_simulation(PROBS, PAYOUTS, T, beta)[0], thompson(PROBS, PAYOUTS, T, beta)[0]

(37.64229538693381, 42.607826996686484)